
# ガウス過程の概要

多くの場合、機械学習はデータからパラメータを推定することになります。これらのパラメータは多くの場合、ニューラル ネットワークの重みなどのように、比較的解釈が困難です。対照的に、ガウス プロセスは、データに適合する可能性のある関数の高レベルの特性について直接推論するためのメカニズムを提供します。たとえば、これらの関数が急速に変化するか、周期的であるか、条件付きの独立性が含まれるか、または翻訳の不変性が含まれるかを把握できる場合があります。ガウス プロセスを使用すると、データに適合する関数値のガウス分布を直接指定することで、これらの特性をモデルに簡単に組み込むことができます。

いくつかの例から始めて、ガウス プロセスがどのように動作するかを感じてみましょう。

入力 $x$ によってインデックス付けされた回帰ターゲット (出力) $y$ の次のデータセットを観察するとします。一例として、目標は二酸化炭素濃度の変化であり、入力はこれらの目標が記録された時間である可能性があります。データにはどのような特徴がありますか?どのくらいの速さで変化しているように見えますか？データポイントは定期的に収集されていますか? それとも入力が欠落していませんか?欠落している領域を埋める、または $x=25$ まで予測することをどのように想像しますか?

![](../img/gp-observed-data.svg)

データをガウス プロセスに適合させるには、まず、どのタイプの関数が合理的であると考えられるかについて事前分布を指定します。ここでは、ガウス プロセスからのいくつかのサンプル関数を示します。この前例は合理的だと思われますか?ここで、データセットに適合する関数を探しているのではなく、入力に応じて変化する速度など、解の適切な高レベルのプロパティを指定していることに注意してください。このノートブックのすべてのプロットを再現するためのコードが、事前分布と推論に関する次のノートブックで示されることに注意してください。 

![](../img/gp-sample-prior-functions.svg)

データを条件付けすると、データに適合する可能性のある関数の事後分布を推測する前にこれを使用できます。ここでは事後関数の例を示します。 

![](../img/gp-sample-posterior-functions.svg)

これらの関数はそれぞれデータと完全に一致しており、各観測を通じて完全に実行されていることがわかります。これらの事後サンプルを使用して予測を行うには、事後サンプルから考えられるすべてのサンプル関数の値を平均して、以下の濃い青色の曲線を作成します。この期待値を計算するために実際には無限の数のサンプルを取得する必要はないことに注意してください。後で説明するように、期待値は閉じた形式で計算できます。 

![](../img/gp-posterior-samples.svg)

また、予測にどの程度自信を持つべきかを知るために、不確実性の表現も必要になる場合があります。直観的には、サンプル事後関数のばらつきが大きい場合には、より多くの不確実性が生じるはずです。これは、真の関数が取り得る値がさらに多くあることを示しているからです。このタイプの不確実性は*認識的不確実性*と呼ばれ、情報の欠如に伴う*還元可能な不確実性*です。より多くのデータを取得するにつれて、観察したものと一致する解決策はますます少なくなるため、この種の不確実性はなくなります。事後平均と同様に、事後分散 (事後におけるこれらの関数の変動) を閉じた形式で計算できます。陰影を付けると、平均値の両側に事後標準偏差の 2 倍が表示され、任意の入力 $x$ に対する関数の真の値を 95% の確率で含む*信頼区間*が作成されます。 

![](../img/gp-posterior-samples-95.svg)

事後サンプルを削除し、データ、事後平均、および 95% の信頼できるセットを単純に視覚化すると、プロットはいくらかきれいに見えます。認識論的不確実性の特性である、データから不確実性がどのように離れていくかに注目してください。 

![](../img/gp-point-predictions.svg)

データのフィッティングに使用したガウス プロセスのプロパティは、*カーネル*とも呼ばれる、いわゆる*共分散関数*によって強力に制御されます。使用した共分散関数は*RBF (Radial Basis Function) カーネル*と呼ばれ、その形式は $$ k_{\text{RBF}}(x,x&#39;) = \mathrm{Cov}(f(x),f( x&#39;)) = a^2 \exp\left(-\frac{1}{2\ell^2}||x-x&#39;||^2\right) $$

このカーネルの*ハイパーパラメータ*は解釈可能です。*振幅*パラメータ $a$ は関数が変化する垂直スケールを制御し、*長さスケール*パラメータ $\ell$ は関数の変動率 (揺れ) を制御します。 $a$ が大きいほど関数値が大きくなり、$\ell$ が大きいほど関数の変化が遅くなります。 $a$ と $\ell$ を変化させると、サンプルの事前関数と事後関数に何が起こるかを見てみましょう。

*長さのスケールは、* GP の予測と不確実性に特に顕著な影響を与えます。 $||x-x&#39;|| で= \ell$ 、関数値のペア間の共分散は $a^2\exp(-0.5)$ です。 $\ell$ よりも大きな距離では、関数値の値はほぼ無相関になります。これは、点 $x_*$ で予測を行いたい場合、$||x-x&#39;||&gt;\ell$ のような入力 $x$ を持つ関数値は予測に強い影響を与えないことを意味します。 。

長さスケールの変更がサンプルの事前関数と事後関数、および信頼できるセットにどのような影響を与えるかを見てみましょう。上記の近似では、$2$ の長さスケールを使用します。 $\ell = 0.1, 0.5, 2, 5, 10$ を考えてみましょう。 $0.1$ という長さスケールは、検討している入力ドメインの範囲である $25$ に比べて非常に小さいです。たとえば、$x=5$ と $x=10$ での関数の値には、そのような長さスケールでは基本的に相関関係がありません。一方、$10$ の長さスケールの場合、これらの入力における関数値は高度に相関します。次の図では縦軸のスケールが変化していることに注意してください。 

![](../img/gp-priorpoint1.svg)![](../img/gp-postpoint1.svg)

![](../img/gp-priorpoint5.svg)![](../img/gp-postpoint5.svg)

![](../img/gp-prior2.svg)![](../img/gp-post2.svg)

![](../img/gp-prior5.svg)![](../img/gp-post5.svg)

長さのスケールが増加するにつれて、関数の「揺れ」が減少し、不確実性が減少することに注目してください。長さスケールが小さい場合、データポイントから関数値に関する情報が少なくなるため、データから遠ざかるにつれて不確実性が急速に増加します。

ここで、長さスケールを $2$ に固定したまま、振幅パラメータを変更してみましょう。垂直スケールは前のサンプルでは固定され、後のサンプルでは変化するため、関数のスケールの増加とデータへの適合の両方が明確に確認できることに注意してください。 

![](../img/gp-priorap1.svg)![](../img/gp-postapoint1.svg)

![](../img/gp-priora2.svg)![](../img/gp-posta2.svg)

![](../img/gp-priora8.svg)![](../img/gp-posta8.svg)

振幅パラメータは関数のスケールには影響しますが、変動率には影響しないことがわかります。この時点で、手順の汎化パフォーマンスは、これらのハイパーパラメーターの適切な値に依存するという感覚もあります。 $\ell=2$ と $a=1$ の値は妥当な適合を提供するように見えましたが、他の値の一部はそうではありませんでした。幸いなことに、いわゆる*周辺尤度*を使用して、これらのハイパーパラメータを指定する堅牢かつ自動的な方法があります。これについては、推論に関するノートで戻ります。

では、そもそも GP とは何でしょうか?開始したように、GP は単に、入力 $x_1,\dots,x_n$ のコレクションによってインデックス付けされた関数値 $f(x_1),\dots,f(x_n)$ のコレクションが同時多変量ガウス分布を持っていると言うだけです。この分布の平均ベクトル $\mu$ は、*平均関数*によって与えられ、通常は定数またはゼロと見なされます。この分布の共分散行列は、入力 $x$ のすべてのペアで評価された*カーネル*によって与えられます。

 $$\begin{bmatrix}f(x) \f(x_1) \ \vdots \ f(x_n) \end{bmatrix}\sim \mathcal{N}\left(\mu, \begin{bmatrix}k(x ,x) &amp; k(x, x_1) &amp; \dots &amp; k(x,x_n) \ k(x_1,x) &amp; k(x_1,x_1) &amp; \dots &amp; k(x_1,x_n) \ \vdots &amp; \vdots &amp; \ddots &amp; \vdots \ k(x_n, x) &amp; k(x_n, x_1) &amp; \dots &amp; k(x_n,x_n) \end{bmatrix}\right)$$ :eqlabel: `eq_gp_prior`

方程式 :eqref: `eq_gp_prior` GP 事前を指定します。 $f(x_1)、\dots、f(x_n)$、観測した関数値が与えられた場合、任意の $x$ に対する $f(x)$ の条件付き分布を計算できます。この条件付き分布は*事後*分布と呼ばれ、予測を行うために使用されます。

特に、

 $$f(x) | f(x_1), \dots, f(x_n) \sim \mathcal{N}(m,s^2)$$

どこ

$$m = k(x,x_{1:n}) k(x_{1:n},x_{1:n})^{-1} f(x_{1:n})$$

 $$s^2 = k(x,x) - k(x,x_{1:n})k(x_{1:n},x_{1:n})^{-1}k(x,x_ {1:n})$$

ここで、$k(x,x_{1:n})$ は、$i=1,\dots,n$ および $k について $k(x,x_{i})$ を評価することによって形成される $1 \times n$ ベクトルです。 (x_{1:n},x_{1:n})$ は、$i,j = 1,\dots,n$ について $k(x_i,x_j)$ を評価することによって形成される $n \times n$ 行列です。 $m$ は任意の $x$ の点予測子として使用できるもので、$s^2$ は不確実性のために使用するものです。$f(x)$ が 95% の確率で区間を作成したい場合が間隔内にある場合は、$m \pm 2s$ を使用します。上記のすべての数値の予測平均と不確実性は、これらの方程式を使用して作成されました。観測されたデータ ポイントは $f(x_1)、\dots、f(x_n)$ によって与えられ、予測を行うためにきめ細かい $x$ ポイントのセットを選択しました。

単一のデータポイント $f(x_1)$ を観察し、ある $x$ での $f(x)$ の値を決定したいとします。 $f(x)$ はガウス過程で記述されるため、$(f(x), f(x_1))$ にわたる同時分布はガウス分布であることがわかります。

 $$ \begin{bmatrix} f(x) \ f(x_1) \ \end{bmatrix} \sim \mathcal{N}\left(\mu, \begin{bmatrix} k(x,x) &amp; k(x , x_1) \ k(x_1,x) &amp; k(x_1,x_1) \end{bmatrix} \right) $$

非対角式 $k(x,x_1) = k(x_1,x)$ は、関数の値がどの程度相関しているかを示します --- $f(x)$ が $f(x_1)$ からどの程度強く決定されるかを示します。 $x$ と $x_1$ の間の距離 $||x-x_1||$ に対して大きな長さスケールを使用すると、関数値の相関性が高くなることがすでにわかりました。 $f(x_1)$ から $f(x)$ を決定するプロセスを、関数空間と $f(x_1)、f(x)$ の結合分布の両方で視覚化できます。最初に $k(x,x_1) = 0.9$ および $k(x,x)=1$ のような $x$ を考えてみましょう。これは、$f(x)$ の値が の値と適度に相関していることを意味します。 $f(x_1)$。同時分布では、一定確率の等高線は比較的狭い楕円になります。

 $f(x_1) = 1.2$ を観測したとします。この $f(x_1)$ の値を条件として、密度のプロット上の $1.2$ に水平線を引くと、$f(x)$ の値が $[0.64,1.52 にほぼ制限されていることがわかります。 ]$。また、このプロットを関数空間に描画し、観測点 $f(x_1)$ をオレンジ色で、$f(x)$ のガウス過程予測分布の 1 標準偏差を青色で示し、平均値 $1.08$ 付近を示しています。 。 

![](https://user-images.githubusercontent.com/6753639/206867364-b4707db5-0c2e-4ae4-a412-8292bca4d08d.svg)![](https://user-images.githubusercontent.com/6753639/206867367-3815720c-93c8-4b4b-80e7-296db1d3553b.svg)

ここで、より強い相関関係 $k(x,x_1) = 0.95$ があると仮定します。現在、楕円はさらに狭くなり、$f(x)$ の値は $f(x_1)$ によってさらに強く決定されます。 $1.2$ で水平線を引くと、$f(x)$ の等高線がほぼ $[0.83, 1.45]$ 以内の値をサポートしていることがわかります。繰り返しますが、$1.14$ の平均予測値に関する 1 つの標準偏差を伴う関数空間のプロットも示します。 

![](https://user-images.githubusercontent.com/6753639/206867797-20e42783-31de-4c50-8103-e9441ba6d0a9.svg)![](https://user-images.githubusercontent.com/6753639/206867800-d9fc7add-649d-492c-8848-cab07c8fb83e.svg)

より強い相関関係が存在するため、ガウス プロセスの事後平均予測子が $1.2$ に近づいていることがわかります。また、不確実性 (エラーバー) が若干減少していることもわかります。これらの関数値の間に強い相関があるにもかかわらず、単一のデータ ポイントしか観測していないため、不確実性は依然として非常に大きいです。

この手順により、観測した任意の数の点について、任意の $x$ について $f(x)$ の事後分布を得ることができます。 $f(x_1)、f(x_2)$ を観察するとします。ここで、関数空間内の特定の $x=x&#39;$ における $f(x)$ の事後分布を視覚化します。 $f(x)$ の正確な分布は、上記の方程式で求められます。 $f(x)$ は平均を伴うガウス分布です

$$m = k(x,x_{1:3}) k(x_{1:3},x_{1:3})^{-1} f(x_{1:3})$$

と分散

$$s^2 = k(x,x) - k(x,x_{1:3})k(x_{1:3},x_{1:3})^{-1}k(x,x_ {1:3})$$

この入門ノートでは、*ノイズのない*観測を検討してきました。これからわか​​るように、観測ノイズが含まれやすいです。データが潜在ノイズのない関数 $f(x)$ と分散 $\ を伴う iid ガウス ノイズ $\epsilon(x) \sim \mathcal{N}(0,\sigma^2)$ から生成されたと仮定すると、 sigma^2$ の場合、共分散関数は単に $k(x_i,x_j) \to k(x_i,x_j) + \delta_{ij}\sigma^2$ になります。ここで $\delta_{ij} = 1$ の場合それ以外の場合は i=j$、$0$。

私たちは、ガウス プロセスを使用して事前および事後解を指定する方法と、カーネル関数がこれらの解の特性にどのような影響を与えるかについて、すでにある程度の直感を持ち始めています。以下のノートブックでは、事前にガウス プロセスを指定する方法、さまざまなカーネル関数を導入および導出する方法を正確に示し、その後、カーネル ハイパーパラメーターを自動的に学習し、予測を行うために事後ガウス プロセスを形成する方法の仕組みを説明します。 「関数上の分布」などの概念に慣れるには時間と練習が必要ですが、GP 予測方程式を見つける実際の仕組みは実際には非常に単純であり、これらの概念を直感的に理解するための練習を容易に行うことができます。 。

## まとめ

一般的な機械学習では、いくつかの自由パラメータ (ニューラル ネットワークとその重みなど) を使用して関数を指定し、解釈できない可能性があるそれらのパラメータの推定に重点を置きます。代わりに、ガウス プロセスを使用すると、関数上の分布について直接推論するため、解の高レベルの特性について推論できるようになります。これらのプロパティは共分散関数 (カーネル) によって制御され、多くの場合、高度に解釈可能なハイパーパラメーターがいくつか含まれています。これらのハイパーパラメータには、関数の速度 (動きの速さ) を制御する*length-scale*が含まれます。もう 1 つのハイパーパラメータは振幅で、関数が変化する垂直スケールを制御します。データに適合できるさまざまな関数を表現し、それらをすべて組み合わせて予測分布を作成するのは、ベイズ手法の特徴です。データから遠く離れた可能な解決策の間には大きなばらつきがあるため、データから離れるにつれて不確実性が直感的に増大します。

ガウス プロセスは、すべての可能な関数値にわたる多変量正規 (ガウス) 分布を指定することによって、関数上の分布を表します。ガウス分布を簡単に操作して、他の値のセットの値に基づいて 1 つの関数値の分布を見つけることができます。言い換えれば、一連の点を観察すると、これらの点に条件を付けて、他の入力における関数の値の分布を推測することができます。これらの点間の相関をどのようにモデル化するかは共分散関数によって決定され、ガウス プロセスの一般化特性を定義します。ガウス プロセスに慣れるには時間がかかりますが、扱いやすく、多くのアプリケーションがあり、ニューラル ネットワークなどの他のモデル クラスを理解して開発するのに役立ちます。

## 演習
1. 認識論的な不確実性と観察の不確実性の違いは何ですか?
1. 変動率と振幅以外に考慮すべき関数の特性は何ですか?また、それらの特性を持つ関数の実例は何ですか?
1. 私たちが検討した RBF 共分散関数は、観測値間の共分散 (および相関) が入力空間 (時間、空間的位置など) での距離に応じて減少することを示しています。これは合理的な仮定ですか?なぜ、あるいはなぜそうではないのでしょうか？
1.  2 つのガウス変数の合計はガウスですか? 2 つのガウス変数の積はガウスですか? (a,b) が結合ガウス分布を持つ場合、a|b (a が与えられた b) はガウス分布ですか?ガウシアンですか？
1.  $f(x_1) = 1.2$ でデータ点を観測する演習を繰り返しますが、さらに $f(x_2) = 1.4$ を観測すると仮定します。 $k(x,x_1) = 0.9$、$k(x,x_2) = 0.8$ とします。 $f(x_1)$ だけを観測していたときよりも、$f(x)$ の値について多かれ少なかれ確信が持てるようになるでしょうか?現在の $f(x)$ の値の平均値と 95% の信頼できる値はいくらですか?
1. 観測ノイズの推定値を増やすと、グラウンド トゥルース関数の長さスケールの推定値が増加するか減少すると思いますか?
1. データから離れるにつれて、予測分布の不確実性がある程度まで増加し、その後増加が停止したとします。なぜそのようなことが起こるのでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/12115)
